# Import libraries and load data

In [261]:
import matplotlib
import pandas as pd
import requests
import io
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import re
from datetime import datetime


%matplotlib inline
    


%matplotlib inline
url1 = "https://raw.githubusercontent.com/docju/capstone2/main/spamraw.csv"
download1 = requests.get(url1).content


df = pd.read_csv(io.BytesIO(download1))
print(df.head(10))

   type                                               text
0   ham  Hope you are having a good week. Just checking in
1   ham                            K..give back my thanks.
2   ham        Am also doing in cbe only. But have to pay.
3  spam  complimentary 4 STAR Ibiza Holiday or £10,000 ...
4  spam  okmail: Dear Dave this is your final notice to...
5   ham  Aiya we discuss later lar... Pick u up at 4 is...
6   ham                             Are you this much buzy
7   ham                    Please ask mummy to call father
8  spam  Marvel Mobile Play the official Ultimate Spide...
9   ham     fyi I'm at usf now, swing by the room whenever


# Explore the data


In [46]:
df.shape

(5559, 2)

In [4]:
df.type.value_counts()

ham     4812
spam     747
Name: type, dtype: int64

In [263]:
# Check for duplicates
df[df.duplicated()].shape

(403, 2)

In [48]:
df=df.drop_duplicates(['type','text'],keep='first')

print(df.shape)

(5156, 2)


In [49]:
df.type.value_counts()

ham     4503
spam     653
Name: type, dtype: int64

# Clean the data

## Remove URL, email, phone, and currency

In [260]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
email_regex='^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})$'
#obtained from stack overflow
phone_regex='[\+]?[0-9 ]{11,14}'
#'((((\(?0\d{4}\)?\s?\d{3}\s?\d{3})|(\(?0\d{3}\)?\s?\d{3}\s?\d{4})|(\(?0\d{2}\)?\s?\d{4}\s?\d{4}))(\s?\(\d{4}|\d{3}))?)|((\+44\s?7\d{3}|\(?07\d{3}\)?)\s?\d{3}\s?\d{3})|((((\+44\s?\d{4}|\(?0\d{4}\)?)\s?\d{3}\s?\d{3})|((\+44\s?\d{3}|\(?0\d{3}\)?)\s?\d{3}\s?\d{4})|((\+44\s?\d{2}|\(?0\d{2}\)?)\s?\d{4}\s?\d{4}))(\s?\(\d{4}|\d{3}))?$'
currency_regex='[\$|£|€][0-9\,]*\.?\d{2}'#'|[\$|£|€]+/-?\d+(,\d{3})*(\.\d{1,2})?$/'
#'[\$|£|€]\d*\.?\d{2}'
# Write a function that performing basic cleaning on the text data by replacing links, phone numbers, email addresses and currency amounts
def clean_text(text):
    '''
    Cleans the text data by replacing URLs, @-replies, hashtags, and emoji with text placeholders.
    
    INPUT:
    text - (str) text data to be cleaned
    
    OUTPUT:
    text - (str) cleaned text data with placeholders
    '''
    
    # Replace URLs
#    detected_urls = re.findall(url_regex, text)
#    for url in detected_urls:
#        text = text.replace(url, "urlplaceholder")
        
    #Replace phone numbers
    detected_phone = re.findall(phone_regex, text)
    #for i in range(0,len(detected_phone)):
    for phone in detected_phone:
           # if len(phone)>0:
        text = text.replace(phone, " phoneplaceholder ")
    # Replace money amounts
    
    #detected_currency=[]
    detected_currency = re.findall(currency_regex, text)
    for amount in detected_currency:
            text = text.replace(amount," cash_amount ")

    detected_email = re.findall(email_regex, text)
    for amount in detected_email:
        text = text.replace(amount," email_address ")
    
    return text

# Apply the text cleaning to our text column
df["clean_text"] = df["text"].apply(lambda x: clean_text(x))

    
df[(df['text']!=df['clean_text']) & (df['type']=='ham')]


,type,text,clean_text
510,ham,"YEH I AM DEF UP4 SOMETHING SAT,JUST GOT PAYED2...","YEH I AM DEF UP4 SOMETHING SAT,JUST GOT PAYED2..."
802,ham,This is ur face test ( 1 2 3 4 5 6 7 8 9 # )...,This is ur face test ( phoneplaceholder 8 9 ...
909,ham,Hey pple...$700 or $900 for 5 nights...Excelle...,Hey pple... cash_amount or cash_amount for ...
1019,ham,Hi this is yijue... It's regarding the 3230 te...,Hi this is yijue... It's regarding the 3230 te...
1950,ham,MY NO. IN LUTON 0125698789 RING ME IF UR AROUN...,MY NO. IN LUTON phoneplaceholder RING ME IF UR...
2485,ham,Hey...Great deal...Farm tour 9am to 5pm $95/pa...,Hey...Great deal...Farm tour 9am to 5pm cash_...
3157,ham,"Hi Jon, Pete here, Ive bin 2 Spain recently & ...","Hi Jon, Pete here, Ive bin 2 Spain recently & ..."
3319,ham,It's Ã© only $140 ard...Ã‰ rest all ard $180 a...,It's Ã© only cash_amount ard...Ã‰ rest all a...
4473,ham,Your bill at 3 is £33.65 so thats not bad!,Your bill at 3 is cash_amount so thats not bad!
4906,ham,Hmm...Bad news...Hype park plaza $700 studio t...,Hmm...Bad news...Hype park plaza cash_amount ...


In [250]:
re.findall(phone_regex, '+447792 893 772')

['+447792 893 772']